In [1]:
import psycopg2
import boto3
import pandas as pd
from sqlalchemy import create_engine

host = 'database-sparkathon.cm6vbnnsye5g.us-east-1.rds.amazonaws.com'
port = 5432
database = 'postgres'
user = 'postgres'
password = "1QCF03Azp9wUQ3EfYeqI"

engine = create_engine(
    f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}",
    connect_args={"sslmode": "require"}
)

In [2]:
sp_core_base_query = """
select *
from
data_dev.sp_komodo_core_base
"""

sp_core_df = pd.read_sql(sp_core_base_query, engine)
sp_core_df.head()

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI


In [3]:
sp_query = """
select *
from
data_dev.sp_data
"""

sp_data_df = pd.read_sql(sp_query, engine)
sp_data_df.head()

,patient_id,referral_date,patient_gender,patient_age,patient_type,rx_id,rx_written_date,ship_date,hub_enrollment_status,enrollment_number,...,final_patient_status,final_case_type,is_dummy_row,record_id,prescriber_zip,prescriber_npi,reopen_flag,bpg_alert_flag,mcoid_mmit,mconame_mmit
0,79540079,2025-08-12,M,31,Adult,NaN,NaT,45895,None,67,...,Active,Commercial,No,PN288502530856-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
1,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-09-04 00:00:00,None,67,...,Active,Commercial,No,PN288554630363-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
2,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-10-03 00:00:00,None,67,...,Active,Commercial,No,PN288796330363-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
3,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-10-30 00:00:00,None,67,...,Active,Commercial,No,PN290152420210-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
4,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-09-05 00:00:00,None,67,...,Active,Commercial,No,PN290528880910-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE


In [4]:
mx_query = """
select *
from
data_dev.mx_core_table
"""

mx_df = pd.read_sql(mx_query, engine)
mx_df.head()

,medical_event_id,patient_id,service_date,service_to_date,procedure_code,procedure_code_type,modifiers,ndc_11,units,unit_type,...,revenue_code,bill_type_code,visit_type,kh_plan_id,diagnosis_codes,event_source,billing_npi_confidence,run_id,created_at,updated_at
0,663e4354f016ae553d173c57976ebe21eb4286ec516f26...,WLC2PZ7D,2024-05-01,2024-05-01,None,None,None,None,0,None,...,0258,111,INPATIENT,7286,|G40909|C712|F1010|F418|F1290|F17200|Z86711|Z9...,INSTITUTIONAL,A - KNOWN,None,None,None
1,975fd956c2aa3b3390293031b5650f1a41f70a77d0edfe...,JXP5L9HT,2023-09-05,2023-09-05,None,None,None,None,1,None,...,0430,117,INPATIENT,8975,|D320|G936|G935|D649|F32A|R001|D72829|N939|T38...,INSTITUTIONAL,A - KNOWN,None,None,None
2,d97f25fe58b765311ed1a948204336f6972277f97459c8...,VBJQD5WM,2024-10-03,2024-10-03,83735,CPT,None,None,None,None,...,None,131,OUTPATIENT,9378,|G9389|R531|Z4682|Z4659|G919|,INSTITUTIONAL,A - KNOWN,None,None,None
3,894c9a71a02083611e2001bb446344cc1cd9ce93738dca...,BR92EMW8,2024-09-12,2024-09-12,97110,CPT,|GP|,None,1,None,...,None,None,OUTPATIENT,6109,|D496|C717|M6281|R260|,PROFESSIONAL,A - KNOWN,None,None,None
4,bfea0b3b111bb3c342cddfe93365959bbf5d1109eb1ddf...,B1K0VVWZ,2024-09-30,2024-09-30,70553,CPT,|26|GC|,None,10,None,...,None,None,OUTPATIENT,7653,|G911|G914|,PROFESSIONAL,A - KNOWN,None,None,None


DIAGNOSIS FUNCTIONS AND FLAG BUILDING

In [340]:
sp_diagnosis_df = sp_core_df.copy()

In [341]:
def get_icd_code_from_data(patient_id, sp_data):
    icd_codes_per_patient: list[str] = (
                                            sp_data.loc[sp_data['patient_id'] == patient_id, 'primary_icd_code']
                                            .dropna()
                                            .unique()
                                        )

    for code in icd_codes_per_patient:
        if code.startswith('C71'):
            return code.replace('.', '')

sp_diagnosis_df['icd_code'] = sp_diagnosis_df['sp_patient_id'].apply(get_icd_code_from_data, args=(sp_data_df,))

In [342]:
import re

def get_icd_code_from_claims(row, claims):
    patient_id = row['matched_patient_id']
    sp_diag_code = row['icd_code']

    if pd.isna(patient_id) or pd.isna(sp_diag_code):
        return None

    sp_diag_code = str(sp_diag_code)
    parent = sp_diag_code[:3] 

    dx_values = (
        claims.loc[claims['patient_id'] == patient_id, 'diagnosis_codes']
        .dropna()
        .unique()
    )

    for dx in dx_values:
        dx_str = str(dx)
        if sp_diag_code in dx_str:
            return sp_diag_code

    pattern = re.compile(rf"{re.escape(parent)}(?:\.\d+|\d+)?")
    for dx in dx_values:
        dx_str = str(dx)
        m = pattern.search(dx_str)
        if m:
            return m.group(0).replace('.', '')

    return None

sp_diagnosis_df['icd_code_claims'] = sp_diagnosis_df.apply(
    get_icd_code_from_claims,
    axis=1,
    args=(mx_df,)
)

In [343]:
def generate_diagnosis_flags(row):
    exact_sp_diag_code = row['icd_code']
    exact_claims_diag_code = row['icd_code_claims']

    if pd.isna(exact_claims_diag_code) or pd.isna(exact_sp_diag_code):
        return 0, 0

    parent_sp_diag_code = str(exact_sp_diag_code)[:3]
    parent_claims_diag_code = str(exact_claims_diag_code)[:3]

    exact_match = 1 if exact_sp_diag_code == exact_claims_diag_code else 0
    parent_match = 1 if parent_sp_diag_code == parent_claims_diag_code else 0

    return parent_match, exact_match

sp_diagnosis_df[['parent_diag_code_flag', 'exact_diag_code_flag']] = sp_diagnosis_df.apply(
    generate_diagnosis_flags,
    axis=1,
    result_type='expand'
)
    

In [344]:
def calculate_diag_freq(row, mx_df):
    claims_patient_id = row['matched_patient_id']
    exact_diag_code_flag = row['exact_diag_code_flag']
    icd_code_claims = row['icd_code_claims']

    freq = 0

    if exact_diag_code_flag == 1 and pd.notna(claims_patient_id) and pd.notna(icd_code_claims):
        filtered_df = mx_df.loc[mx_df['patient_id'] == claims_patient_id]
        icd_codes = filtered_df['diagnosis_codes'].dropna().astype(str).values

        target = str(icd_code_claims)
        for code in icd_codes:
            if code.find(target) != -1:
                freq += 1

    return freq

freqs = []
for _, row in sp_diagnosis_df.iterrows():
    freqs.append(calculate_diag_freq(row, mx_df))

sp_diagnosis_df['diag_freq'] = freqs

In [345]:
from pandas import DataFrame
from datetime import datetime, timedelta

def generate_diagnosis_lookback(row, sp_data: DataFrame, mx_df):
    sp_patient_id = row['sp_patient_id']
    claims_patient_id = row['matched_patient_id']
    icd_code = row['icd_code']
    exact_diag_code_flag = row['exact_diag_code_flag']

    if exact_diag_code_flag != 1 or pd.isna(sp_patient_id) or pd.isna(claims_patient_id) or pd.isna(icd_code):
        return 0, 0, 0
    
    sp_ref_dates = pd.to_datetime(
        sp_data.loc[sp_data['patient_id'] == sp_patient_id, 'referral_date'],
        errors='coerce'
    )

    max_referral_date = sp_ref_dates.max()

    if pd.isna(max_referral_date):
        return 0, 0, 0

    date_90_days_before = max_referral_date - timedelta(days=90)
    date_180_days_before = max_referral_date - timedelta(days=180)
    date_360_days_before = max_referral_date - timedelta(days=360)

    claims = mx_df.loc[mx_df['patient_id'] == claims_patient_id].copy()
    claims['service_date'] = pd.to_datetime(claims['service_date'], errors='coerce')

    claims = claims.dropna(subset=['service_date'])

    claims_before_90_days = claims.loc[claims['service_date'] >= date_90_days_before]
    claims_before_180_days = claims.loc[claims['service_date'] >= date_180_days_before]
    claims_before_360_days = claims.loc[claims['service_date'] >= date_360_days_before]

    icd_code_str = str(icd_code)

    mask_90 = claims_before_90_days['diagnosis_codes'].astype(str).str.contains(icd_code_str, na=False)
    mask_180 = claims_before_180_days['diagnosis_codes'].astype(str).str.contains(icd_code_str, na=False)
    mask_360 = claims_before_360_days['diagnosis_codes'].astype(str).str.contains(icd_code_str, na=False)

    return int(mask_90.sum()), int(mask_180.sum()), int(mask_360.sum()) 

sp_diagnosis_df[['90_days_lookback_freq', '180_days_lookback_freq', '360_days_lookback_freq']] = sp_diagnosis_df.apply(
    generate_diagnosis_lookback,
    axis=1,
    args=(sp_data_df, mx_df),
    result_type='expand'
)

In [346]:
def generate_lookback_frequency(row, mx_df):
    icd_code = row['icd_code']
    exact_diag_code_flag = row['exact_diag_code_flag']
    claims_patient_id = row['matched_patient_id']

    if not (exact_diag_code_flag == 1 and pd.notna(icd_code) and pd.notna(claims_patient_id)):
        return None

    claims_filtered = mx_df.loc[
                    (mx_df['patient_id'] == claims_patient_id) & 
                    (mx_df['diagnosis_codes']
                    .astype(str)
                    .str
                    .contains(icd_code, na=False))].copy()

    if claims_filtered.empty:
        return None

    claims_filtered['service_date'] = pd.to_datetime(claims_filtered['service_date'], errors='coerce')
    claims_filtered = claims_filtered.dropna(subset=['service_date'])

    first_claim = claims_filtered['service_date'].min()
    last_claim = claims_filtered['service_date'].max()

    time_period = (last_claim - first_claim).days

    return int(time_period)

sp_diagnosis_df['first_last_claim_lookback'] = sp_diagnosis_df.apply(
    generate_lookback_frequency,
    axis=1,
    args=(mx_df,)
)

In [347]:
sp_diagnosis_df = sp_diagnosis_df.drop_duplicates(subset=['sp_prescriber_npi', 'sp_patient_id', 'matched_patient_id'])
sp_diagnosis_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,icd_code,icd_code_claims,parent_diag_code_flag,exact_diag_code_flag,diag_freq,90_days_lookback_freq,180_days_lookback_freq,360_days_lookback_freq,first_last_claim_lookback
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,C719,C719,1,1,57,23,37,57,283.0
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,C719,C719,1,1,89,2,2,9,679.0
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,C719,C719,1,1,5,0,0,5,0.0
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,C719,C719,1,1,41,2,3,41,158.0
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,C719,C719,1,1,390,29,29,48,590.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,C719,C719,1,1,5,5,5,5,105.0
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,C719,C719,1,1,21,1,1,9,721.0
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,C719,C719,1,1,2,0,0,0,0.0
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,C719,None,0,0,0,0,0,0,NaN


THERAPY AREA FUNCTIONS AND FLAG BUILDING

In [348]:
treatment_codes = {
    "radiation": [
        "61796","61797","61798","61799","61800",
        "77261","77262","77263","77280","77285","77290","77293","77295",
        "77300","77301","77306","77307","77316","77317","77318",
        "77321","77331","77332","77333","77334","77336","77338",
        "77370","77371","77372","77373","77385","77386","77387",
        "77401","77402","77407","77412","77417","77424","77425",
        "77427","77431","77470",
        "77520","77522","77523","77525",
        "77761","77762","77763","77767","77768","77770","77771","77772",
        "0395T",
        "G0339","G0340",
        "G6001","G6002","G6003","G6004","G6005","G6006","G6007",
        "G6008","G6009","G6010","G6011","G6012","G6013","G6014",
        "G6015","G6016","G6017",
        "D0012ZZ","D0Y07ZZ","D0010ZZ","D0000ZZ","D0002ZZ","D0001ZZ",
        "D0Y0FZZ","D0Y17ZZ","D0006ZZ","D0011ZZ","D0003ZZ","D0005ZZ"
    ],

    "chemotherapy": [
        "96400","96408","96412","96414","96445","96545","99555",
        "96360","96361","96365","96366","96367","96368","96369",
        "96370","96371","96372","96373","96374","96375","96376","96379",
        "96409","96411","96413","96415","96417","96549",
        "G0498","G8928","G8929",
        "J8999",
        "J9000-J9999", 
        "Q0081","Q0084"
    ],

    "surgery": [
        "61790","81463",
        "61750","61751",
        "61510","61518","61519","61520","61512","61521",
        "61526","61530"
    ],

    "testing": [
        "88341","88342",
        "88344","88360",
        "88361","81401",
        "81479","81210",
        "81445","81425"
    ]
}


In [349]:
sp_therapy_df = pd.read_sql("select * from data_dev.sp_komodo_core_base", engine)
sp_therapy_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI
...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA


In [350]:
def generate_treatment_flag(row, treatment_codes: list[str], mx: DataFrame):
    mx_patient_id = row['matched_patient_id']

    if pd.isna(mx_patient_id):
        return 0, 0

    codes = [str(c) for c in treatment_codes]

    mx_filtered = mx.loc[
        (mx['patient_id'] == mx_patient_id) &
        (mx['procedure_code'].astype(str).isin(codes))
    ]

    if mx_filtered.empty:
        return 0, 0

    return 1, int(len(mx_filtered))

In [351]:
for treatment, codes in treatment_codes.items():
    sp_therapy_df[[f'{treatment}_flag', f'{treatment}_freq']] = sp_therapy_df.apply(
        generate_treatment_flag,
        axis=1,
        args=(codes, mx_df),
        result_type='expand'
    )

In [352]:
sp_therapy_df = sp_therapy_df.drop_duplicates(subset=['sp_prescriber_npi', 'sp_patient_id', 'matched_patient_id'])
sp_therapy_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,radiation_flag,radiation_freq,chemotherapy_flag,chemotherapy_freq,surgery_flag,surgery_freq,testing_flag,testing_freq
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,1,21,0,0,0,0,1,3
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,0,0,0,0,0,0,0,0
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,1,64,1,81,1,1,1,4
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,1,111,1,11,1,4,1,7
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,1,55,0,0,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,0,0,0,0,1,1,1,2
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,0,0,0,0,0,0,0,0
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,0,0,0,0,1,6,1,14
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,0,0,1,1,1,1,1,2


In [353]:
px_query = """
select *
from
data_dev.px_core_table
"""

px_df = pd.read_sql(px_query, engine)
px_df = px_df[(px_df['brand_name'] == 'MODEYSO') & (px_df['generic_name'] == 'DORDAVIPRONE HCL')]
px_df.head()

,pharmacy_event_id,patient_id,fill_date,pharmacy_channel,pharmacy_npi,prescriber_npi,ndc_11,brand_name,generic_name,route,...,daw_code,date_prescription_written,primary_kh_plan_id,secondary_kh_plan_id,patient_responsibility,patient_oop,transaction_source_type,lag_source_type,cob_availability_source_type,prescriber_confidence
54,05115fb10082f05fa570482f3c510e9fd0d9e52ff82dc5...,L28W0Q4X,2025-09-25,RETAIL,1356973887,1801403290,68727025001,MODEYSO,DORDAVIPRONE HCL,ORAL,...,0,2025-09-25,213,18663,None,None,PAID ONLY,LOW,NOT AVAILABLE,A - KNOWN
820,d3a1e642c6e8f27fa58d600dc4f7404ac7e35e1fcfb045...,4MW6BP9B,2025-08-19,SPECIALTY,1467941302,1033301411,68727025001,MODEYSO,DORDAVIPRONE HCL,ORAL,...,0,2025-08-11,32,18663,None,None,PAID ONLY,LOW,NOT AVAILABLE,A - KNOWN
2040,01858badf4a61124818a91ad4b86cdd9fdd121cba15c94...,RZ2YC5VN,2025-08-13,SPECIALTY,1679618151,1336307818,68727025001,MODEYSO,DORDAVIPRONE HCL,ORAL,...,0,2025-08-11,213,18663,None,None,PAID ONLY,LOW,NOT AVAILABLE,A - KNOWN
2371,b9fe054da7b67920e809867356d5f26dac5398d155f2c9...,4MW6BP9B,2025-09-24,SPECIALTY,1467941302,1033301411,68727025001,MODEYSO,DORDAVIPRONE HCL,ORAL,...,0,2025-08-06,32,18663,None,None,PAID ONLY,LOW,NOT AVAILABLE,A - KNOWN
2635,6f0128d86195ee04d5ee341a2478dbc756ef969400e739...,YFN4W5SX,2025-08-16,SPECIALTY,1437577988,1033301411,68727025001,MODEYSO,DORDAVIPRONE HCL,ORAL,...,0,2025-07-30,74,18663,None,None,PAID ONLY,LOW,NOT AVAILABLE,A - KNOWN


In [354]:
sp_df = pd.read_sql(sp_query, engine)
sp_df = sp_df[sp_df['productname'] == 'MODEYSO']

In [355]:
sp_core_df = pd.read_sql(sp_core_base_query, engine)
sp_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI
...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA


DISPENSE AND UTILIZATION

In [356]:
sp_df = pd.read_sql(sp_query, engine)

In [357]:
sp_dispense = sp_df[['patient_id', 'rx_written_date', 'ship_date', 'days_supply', 'dispense_quantity']]
sp_dispense

,patient_id,rx_written_date,ship_date,days_supply,dispense_quantity
0,79540079,NaT,45895,NaN,NaN
1,79540079,2025-08-12,2025-09-04 00:00:00,NaN,NaN
2,79540079,2025-08-12,2025-10-03 00:00:00,NaN,NaN
3,79540079,2025-08-12,2025-10-30 00:00:00,NaN,NaN
4,79540079,2025-08-12,2025-09-05 00:00:00,NaN,NaN
...,...,...,...,...,...
4155,9982209982,NaT,None,NaN,NaN
4156,9982209982,2025-11-05,None,NaN,NaN
4157,9982209982,2025-11-05,None,NaN,NaN
4158,9982209982,2025-11-05,None,NaN,NaN


In [358]:
def cleanse_ship_date(ship_date):
    if ship_date is None or pd.isna(ship_date):
        return pd.NaT
    
    ship_date = str(ship_date)

    if '-' in ship_date:
        return pd.to_datetime(ship_date).date()

    converted_date = pd.to_datetime(int(ship_date), origin='1899-12-30', unit='D').date()

    return converted_date

In [359]:
sp_dispense['ship_date'] = sp_dispense['ship_date'].apply(
    cleanse_ship_date
)
sp_dispense['ship_date'] = pd.to_datetime(sp_dispense['ship_date'], errors='coerce')

C:\Users\RuchirTyagi\AppData\Local\Temp\ipykernel_28544\1498183310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_dispense['ship_date'] = sp_dispense['ship_date'].apply(
C:\Users\RuchirTyagi\AppData\Local\Temp\ipykernel_28544\1498183310.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_dispense['ship_date'] = pd.to_datetime(sp_dispense['ship_date'], errors='coerce')


In [360]:
sp_dispense.head()

,patient_id,rx_written_date,ship_date,days_supply,dispense_quantity
0,79540079,NaT,2025-08-26,NaN,NaN
1,79540079,2025-08-12,2025-09-04,NaN,NaN
2,79540079,2025-08-12,2025-10-03,NaN,NaN
3,79540079,2025-08-12,2025-10-30,NaN,NaN
4,79540079,2025-08-12,2025-09-05,NaN,NaN


In [361]:
px_dispense = px_df[['patient_id', 'fill_date', 'date_prescription_written', 'transaction_result', 'days_supply', 'quantity']].copy()
px_dispense

,patient_id,fill_date,date_prescription_written,transaction_result,days_supply,quantity
54,L28W0Q4X,2025-09-25,2025-09-25,REJECTED,28,20
820,4MW6BP9B,2025-08-19,2025-08-11,REJECTED,28,20
2040,RZ2YC5VN,2025-08-13,2025-08-11,REJECTED,28,20
2371,4MW6BP9B,2025-09-24,2025-08-06,PAID,28,20
2635,YFN4W5SX,2025-08-16,2025-07-30,REJECTED,28,12
...,...,...,...,...,...,...
150201,D79VEXMW,2025-09-06,2025-08-28,REJECTED,28,16
151102,CTQYR3B1,2025-09-12,2025-08-28,REJECTED,28,20
151192,01SLEM3L,2025-08-28,2025-07-31,PAID,28,12
151221,L5C26NQH,2025-09-10,2025-09-05,REJECTED,28,20


In [362]:
px_dispense['fill_date'] = pd.to_datetime(px_dispense['fill_date'])
px_dispense['date_prescription_written'] = pd.to_datetime(px_dispense['date_prescription_written'])

px_dispense.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499 entries, 54 to 151241
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   patient_id                 499 non-null    object        
 1   fill_date                  499 non-null    datetime64[ns]
 2   date_prescription_written  482 non-null    datetime64[ns]
 3   transaction_result         499 non-null    object        
 4   days_supply                499 non-null    object        
 5   quantity                   499 non-null    object        
dtypes: datetime64[ns](2), object(4)
memory usage: 27.3+ KB


In [363]:
sp_core_copy = sp_core_df.copy()
sp_core_copy = sp_core_copy.merge(sp_dispense, how='left', left_on='sp_patient_id', right_on='patient_id').drop(columns=['patient_id'])
sp_core_copy = sp_core_copy.merge(px_dispense, how='left', left_on='matched_patient_id', right_on='patient_id').drop(columns=['patient_id'])
sp_core_copy

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,rx_written_date,ship_date,days_supply_x,dispense_quantity,fill_date,date_prescription_written,transaction_result,days_supply_y,quantity
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,NaT,NaT,NaN,NaN,2025-09-18,2025-09-17,PAID,28,20
1,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,NaT,NaT,NaN,NaN,2025-08-21,2025-08-05,PAID,28,20
2,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,NaT,NaN,NaN,2025-09-18,2025-09-17,PAID,28,20
3,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,NaT,NaN,NaN,2025-08-21,2025-08-05,PAID,28,20
4,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,NaT,NaN,NaN,2025-09-18,2025-09-17,PAID,28,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38257,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-16,2025-11-25,14.0,10.0,NaT,NaT,NaN,NaN,NaN
38258,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-03,2025-09-08,NaN,NaN,NaT,NaT,NaN,NaN,NaN
38259,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-03,2025-10-03,NaN,NaN,NaT,NaT,NaN,NaN,NaN
38260,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-03,2025-11-03,28.0,20.0,NaT,NaT,NaN,NaN,NaN


In [364]:
sp_core_copy['fill_date'] = pd.to_datetime(sp_core_copy['fill_date'], errors='coerce')
sp_core_copy['date_prescription_written'] = pd.to_datetime(sp_core_copy['date_prescription_written'], errors='coerce')
sp_core_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38262 entries, 0 to 38261
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   sp_prescriber_npi          38262 non-null  float64       
 1   sp_patient_id              38262 non-null  int64         
 2   sp_state                   38262 non-null  object        
 3   sp_gender                  38262 non-null  object        
 4   sp_age                     38262 non-null  int64         
 5   matched_patient_id         38262 non-null  object        
 6   matched_gender             38262 non-null  object        
 7   matched_age_2025           38262 non-null  int64         
 8   matched_state              38262 non-null  object        
 9   rx_written_date            35320 non-null  datetime64[ns]
 10  ship_date                  7745 non-null   datetime64[ns]
 11  days_supply_x              7834 non-null   float64       
 12  disp

In [365]:
sp_core_copy.head(1)

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,rx_written_date,ship_date,days_supply_x,dispense_quantity,fill_date,date_prescription_written,transaction_result,days_supply_y,quantity
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,NaT,NaT,NaN,NaN,2025-09-18,2025-09-17,PAID,28,20


In [366]:
sp_core_copy = sp_core_copy.drop_duplicates(subset=sp_core_copy.columns.to_list())
sp_core_copy = sp_core_copy.dropna(subset=['rx_written_date'])
sp_core_copy

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,rx_written_date,ship_date,days_supply_x,dispense_quantity,fill_date,date_prescription_written,transaction_result,days_supply_y,quantity
2,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,NaT,NaN,NaN,2025-09-18,2025-09-17,PAID,28,20
3,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,NaT,NaN,NaN,2025-08-21,2025-08-05,PAID,28,20
14,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,NaT,28.0,20.0,2025-09-18,2025-09-17,PAID,28,20
15,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,NaT,28.0,20.0,2025-08-21,2025-08-05,PAID,28,20
16,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-09-17,NaT,28.0,20.0,2025-09-18,2025-09-17,PAID,28,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38257,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-16,2025-11-25,14.0,10.0,NaT,NaT,NaN,NaN,NaN
38258,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-03,2025-09-08,NaN,NaN,NaT,NaT,NaN,NaN,NaN
38259,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-03,2025-10-03,NaN,NaN,NaT,NaT,NaN,NaN,NaN
38260,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,2025-09-03,2025-11-03,28.0,20.0,NaT,NaT,NaN,NaN,NaN


In [367]:
def generate_prescription_flag(row):
    transaction_result = row['transaction_result']
    sp_rx = row['rx_written_date']
    cl_rx = row['date_prescription_written']

    sp_fill = row['ship_date']
    cl_fill = row['fill_date']

    if pd.isna(sp_rx) or pd.isna(cl_rx):
        return -1, -1, -1

    rx_flag = 1 if sp_rx == cl_rx else 0

    if transaction_result != 'PAID':
        return rx_flag, -1, -1

    if pd.isna(sp_fill) or pd.isna(cl_fill):
        return rx_flag, -1, -1
    
    days_lag = (cl_fill - sp_fill).days

    if days_lag < 0:
        days_lag *= -1

    exact_fill_flag = 1 if sp_fill == cl_fill else 0
    lag_fill_flag = 1 if days_lag <= 7 else 0

    return rx_flag, exact_fill_flag, lag_fill_flag 

In [368]:
sp_core_copy[['rx_flag', 'exact_fill_flag', 'lag_fill_flag']] = sp_core_copy.apply(
    generate_prescription_flag,
    axis=1,
    result_type='expand'
)

In [369]:
sp_core_copy.head(1)

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,rx_written_date,...,days_supply_x,dispense_quantity,fill_date,date_prescription_written,transaction_result,days_supply_y,quantity,rx_flag,exact_fill_flag,lag_fill_flag
2,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,...,NaN,NaN,2025-09-18,2025-09-17,PAID,28,20,0,-1,-1


In [370]:
sp_core_copy['days_supply_x'] = pd.to_numeric(sp_core_copy['days_supply_x'], errors='coerce')
sp_core_copy['dispense_quantity'] = pd.to_numeric(sp_core_copy['dispense_quantity'], errors='coerce')
sp_core_copy['days_supply_y'] = pd.to_numeric(sp_core_copy['days_supply_y'], errors='coerce')
sp_core_copy['quantity'] = pd.to_numeric(sp_core_copy['quantity'], errors='coerce')

In [371]:
def generate_quantity_dispensed_flag(row):
    transaction_result = row['transaction_result']

    sp_days_supply = row['days_supply_x']
    sp_dispense_quantity = row['dispense_quantity']

    claims_days_supply = row['days_supply_y']
    claims_dispense_quantity = row['quantity']

    sp_fill = row['ship_date']
    cl_fill = row['fill_date']

    if transaction_result != 'PAID':
        return -1, -1

    #match date
    if (
        pd.isna(sp_days_supply) or pd.isna(claims_days_supply) or pd.isna(claims_days_supply) or pd.isna(claims_dispense_quantity) or pd.isna(sp_fill) or pd.isna(cl_fill)
    ):
        return -1, -1

    cl_fill = cl_fill.date()
    sp_fill = sp_fill.date()

    days_lag = abs((cl_fill - sp_fill).days)
    
    if days_lag <= 7:
        days_supply_flag = 1 if sp_days_supply == claims_days_supply else 0
        quantity_flag = 1 if sp_dispense_quantity == claims_dispense_quantity else 0

        return days_supply_flag, quantity_flag
    
    return 0, 0

In [372]:
sp_core_copy[['days_supply_flag', 'quantity_flag']] = sp_core_copy.apply(
    generate_quantity_dispensed_flag,
    axis=1,
    result_type='expand'
)

In [373]:
sp_core_copy = sp_core_copy.drop_duplicates(subset=['sp_prescriber_npi', 'sp_patient_id', 'matched_patient_id'])

In [304]:
print("Unique SP Patient ID with same written prescription date: ",len(sp_core_copy[sp_core_copy['rx_flag'] == 1]['sp_patient_id'].unique()))
print("Unique Claims Patient ID with same written prescription date: ", len(sp_core_copy[sp_core_copy['rx_flag'] == 1]['matched_patient_id'].unique()))
print("==============================================================================================================================================")
print("Unique SP Patient ID with same Fill date: ", len(sp_core_copy[sp_core_copy['exact_fill_flag'] == 1]['sp_patient_id'].unique()))
print("Unique Claims Patient ID with same Fill date: ", len(sp_core_copy[sp_core_copy['exact_fill_flag'] == 1]['matched_patient_id'].unique()))
print("==============================================================================================================================================")
print("Unique SP Patient ID with lagged Fill date (within 7 days): ", len(sp_core_copy[sp_core_copy['lag_fill_flag'] == 1]['sp_patient_id'].unique()))
print("Unique Claims Patient ID with lagged Fill date (within 7 days): ", len(sp_core_copy[sp_core_copy['lag_fill_flag'] == 1]['matched_patient_id'].unique()))
print("==============================================================================================================================================")
print("Unique SP Patient ID with exact days of supply: ", len(sp_core_copy[sp_core_copy['days_supply_flag'] == 1]['sp_patient_id'].unique()))
print("Unique Claims Patient ID with exact days of supply: ", len(sp_core_copy[sp_core_copy['days_supply_flag'] == 1]['matched_patient_id'].unique()))
print("==============================================================================================================================================")
print("Unique SP Patient ID with exact quantity dispensed: ", len(sp_core_copy[sp_core_copy['quantity_flag'] == 1]['sp_patient_id'].unique()))
print("Unique Claims Patient ID with exact quantity dispensed: ", len(sp_core_copy[sp_core_copy['quantity_flag'] == 1]['matched_patient_id'].unique()))

Unique SP Patient ID with same written prescription date:  74
Unique Claims Patient ID with same written prescription date:  72
Unique SP Patient ID with same Fill date:  4
Unique Claims Patient ID with same Fill date:  4
Unique SP Patient ID with lagged Fill date (within 7 days):  14
Unique Claims Patient ID with lagged Fill date (within 7 days):  14
Unique SP Patient ID with exact days of supply:  4
Unique Claims Patient ID with exact days of supply:  4
Unique SP Patient ID with exact quantity dispensed:  4
Unique Claims Patient ID with exact quantity dispensed:  4


PAYOR

In [374]:
px_payor_query = """
select 
patient_id,
primary_kh_plan_id
from
data_dev.px_core_table
"""

px_payor_df = pd.read_sql(px_payor_query, engine)
px_payor_df.head()

,patient_id,primary_kh_plan_id
0,F22TGCNV,74
1,QT4PX6DP,1312
2,LXPS21SF,74
3,N5QE0WHP,2173
4,L1ZX027S,985


In [375]:
plan_query = """
select
kh_plan_id,
payer_name,
pbm_processor,
insurance_group,
insurance_segment
from
data_dev.komodo_plans
"""

plans_df = pd.read_sql(plan_query, engine)
plans_df.head()

,kh_plan_id,payer_name,pbm_processor,insurance_group,insurance_segment
0,10080,ELIXIR INSURANCE,None,MEDICARE,None
1,10359,CAREPLUS HEALTH PLANS,None,MEDICARE,None
2,5976,HIGHMARK WHOLECARE,None,MEDICARE,ADVANTAGE
3,439,OPTUMRX,OPTUMRX,COMMERCIAL,PBM
4,809,HORIZON BLUE CROSS BLUE SHIELD OF NEW JERSEY,None,None,None


In [376]:
px_payor_df = pd.merge(px_payor_df, plans_df, how='left', left_on='primary_kh_plan_id', right_on='kh_plan_id').drop(columns=['kh_plan_id'])
px_payor_df

,patient_id,primary_kh_plan_id,payer_name,pbm_processor,insurance_group,insurance_segment
0,F22TGCNV,74,CVS CAREMARK,CVS CAREMARK,COMMERCIAL,PBM
1,QT4PX6DP,1312,SILVERSCRIPT,CVS CAREMARK,MEDICARE,PDP
2,LXPS21SF,74,CVS CAREMARK,CVS CAREMARK,COMMERCIAL,PBM
3,N5QE0WHP,2173,CAPITAL BLUECROSS,PRIME THERAPEUTICS,COMMERCIAL,INTEGRATED
4,L1ZX027S,985,HIGHMARK BLUE SHIELD (PA),EXPRESS SCRIPTS,COMMERCIAL,INTEGRATED
...,...,...,...,...,...,...
151461,KBDWFG9X,2678,WELLPOINT (AMERIGROUP),CVS CAREMARK,MEDICAID,MANAGED
151462,EDXGBQV2,483,FLORIDA BLUE,PRIME THERAPEUTICS,COMMERCIAL,INTEGRATED
151463,P4QLQV0E,74,CVS CAREMARK,CVS CAREMARK,COMMERCIAL,PBM
151464,XE3FQTME,962,BLUE MEDICARE RX,CVS CAREMARK,MEDICARE,PDP


In [377]:
sp_payor_query = """
select 
patient_id,
payor_type,
payor_name,
pbm,
plan_name
from
data_dev.sp_data
"""

sp_payor_df = pd.read_sql(sp_payor_query, engine)
sp_payor_df.head()

,patient_id,payor_type,payor_name,pbm,plan_name
0,79540079,Not Mapped,Not Mapped,Not Mapped,Not Mapped
1,79540079,Tricare,Department of Defense,Not Mapped,Tricare
2,79540079,Tricare,Department of Defense,Not Mapped,Tricare
3,79540079,Tricare,Department of Defense,Not Mapped,Tricare
4,79540079,Tricare,Department of Defense,Not Mapped,Tricare


In [378]:
sp_payor_core_df = sp_core_df.copy()
sp_payor_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI
...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA


In [379]:
sp_payor_core_df = pd.merge(sp_payor_core_df, sp_payor_df, how='left', left_on='sp_patient_id', right_on='patient_id').drop(columns=['patient_id'])
sp_payor_core_df.head(3)

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,payor_type,payor_name,pbm,plan_name
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Not Mapped,Not Mapped,Not Mapped,Not Mapped
1,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Commercial,Providence Health,Providence Health,Intel
2,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Commercial,Providence Health,Providence Health,Intel


In [380]:
sp_payor_core_df = sp_payor_core_df.drop_duplicates(subset=sp_payor_core_df.columns.to_list())
sp_payor_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,payor_type,payor_name,pbm,plan_name
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Not Mapped,Not Mapped,Not Mapped,Not Mapped
1,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Commercial,Providence Health,Providence Health,Intel
12,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,Not Mapped,Not Mapped,Not Mapped,Not Mapped
13,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,Commercial,Providence Health,Providence Health,Intel
24,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,Not Mapped,Not Mapped,Not Mapped,Not Mapped
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29820,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,Coupon,All Other Assistance Program,Not Mapped,All Other Voucher
29830,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,Commercial,Express Scripts,Express Scripts,Express Scripts Administered Plans
29834,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,Coupon,All Other Assistance Program,Not Mapped,All Other Voucher
29844,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,Commercial,Express Scripts,Express Scripts,Express Scripts Administered Plans


In [381]:
sp_payor_core_df = pd.merge(sp_payor_core_df, px_payor_df, how='left', left_on='matched_patient_id', right_on='patient_id').drop(columns='patient_id')
sp_payor_core_df = sp_payor_core_df.drop_duplicates(sp_payor_core_df.columns.to_list())
sp_payor_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,payor_type,payor_name,pbm,plan_name,primary_kh_plan_id,payer_name,pbm_processor,insurance_group,insurance_segment
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Not Mapped,Not Mapped,Not Mapped,Not Mapped,393,EXPRESS SCRIPTS,EXPRESS SCRIPTS,COMMERCIAL,PBM
46,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Not Mapped,Not Mapped,Not Mapped,Not Mapped,4275,None,None,DISCOUNT CARD,None
108,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Not Mapped,Not Mapped,Not Mapped,Not Mapped,4145,None,None,SPECIAL PROGRAM,None
131,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Not Mapped,Not Mapped,Not Mapped,Not Mapped,3708,None,OPTUMRX,DISCOUNT CARD,None
134,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,Commercial,Providence Health,Providence Health,Intel,393,EXPRESS SCRIPTS,EXPRESS SCRIPTS,COMMERCIAL,PBM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353375,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,Coupon,All Other Assistance Program,Not Mapped,All Other Voucher,1269,None,None,COMMERCIAL,None
353402,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,Commercial,Express Scripts,Express Scripts,Express Scripts Administered Plans,167,CIGNA,EXPRESS SCRIPTS,COMMERCIAL,INTEGRATED
353435,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,Coupon,All Other Assistance Program,Not Mapped,All Other Voucher,167,CIGNA,EXPRESS SCRIPTS,COMMERCIAL,INTEGRATED
353468,1.992402e+09,1880171880,VA,M,12,H2J20RG9,M,13,VA,Commercial,Express Scripts,Express Scripts,Express Scripts Administered Plans,395,CVS CAREMARK,CVS CAREMARK,COMMERCIAL,PBM


In [382]:
def generate_payor_name_flag(row):
    sp_payor_name = row['payor_name']
    claims_payor_name = row['payer_name']

    sp_payor_name = str(sp_payor_name).lower()
    claims_payor_name = str(claims_payor_name).lower()

    return 1 if sp_payor_name == claims_payor_name else 0

def generate_payor_type_flag(row):
    sp_payor_type = row['payor_type']
    claims_payor_type = row['insurance_group']

    sp_payor_type = str(sp_payor_type).lower()
    claims_payor_type = str(claims_payor_type).lower()

    return 1 if sp_payor_type == claims_payor_type else 0

def generate_pbm_flag(row):
    sp_pbm = row['pbm']
    claims_pbm = row['pbm_processor']

    sp_pbm = str(sp_pbm).lower()
    claims_pbm = str(claims_pbm).lower()

    return 1 if sp_pbm == claims_pbm else 0

In [383]:
sp_payor_core_df['payor_name_flag'] = sp_payor_core_df.apply(
    generate_payor_name_flag,
    axis=1,
    result_type='expand'
)

sp_payor_core_df['payor_type_flag'] = sp_payor_core_df.apply(
    generate_payor_type_flag,
    axis=1,
    result_type='expand'
)

sp_payor_core_df['pbm_flag'] = sp_payor_core_df.apply(
    generate_pbm_flag,
    axis=1, 
    result_type='expand'
)

In [384]:
sp_payor_core_df = sp_payor_core_df.drop_duplicates(subset=['sp_prescriber_npi', 'sp_patient_id', 'matched_patient_id'])

In [307]:
sp_therapy_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,radiation_flag,radiation_freq,chemotherapy_flag,chemotherapy_freq,surgery_flag,surgery_freq,testing_flag,testing_freq
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,1,21,0,0,0,0,1,3
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,0,0,0,0,0,0,0,0
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,1,64,1,81,1,1,1,4
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,1,111,1,11,1,4,1,7
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,1,55,0,0,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,0,0,0,0,1,1,1,2
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,0,0,0,0,0,0,0,0
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,0,0,0,0,1,6,1,14
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,0,0,1,1,1,1,1,2


In [385]:
sp_diagnosis_therapy_df = pd.merge(sp_diagnosis_df, sp_therapy_df, how='left', 
                            left_on=['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state'], 
                            right_on=['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state'])

sp_diagnosis_therapy_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,icd_code,...,360_days_lookback_freq,first_last_claim_lookback,radiation_flag,radiation_freq,chemotherapy_flag,chemotherapy_freq,surgery_flag,surgery_freq,testing_flag,testing_freq
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,C719,...,57,283.0,1,21,0,0,0,0,1,3
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,C719,...,9,679.0,0,0,0,0,0,0,0,0
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,C719,...,5,0.0,1,64,1,81,1,1,1,4
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,C719,...,41,158.0,1,111,1,11,1,4,1,7
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,C719,...,48,590.0,1,55,0,0,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2201,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,C719,...,5,105.0,0,0,0,0,1,1,1,2
2202,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,C719,...,9,721.0,0,0,0,0,0,0,0,0
2203,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,C719,...,0,0.0,0,0,0,0,1,6,1,14
2204,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,C719,...,0,NaN,0,0,1,1,1,1,1,2


In [386]:
sp_core_copy['rx_date_match_flag'] = sp_core_copy.groupby(['sp_patient_id', 'matched_patient_id'])['rx_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_core_copy['exact_fill_date_flag'] = sp_core_copy.groupby(['sp_patient_id', 'matched_patient_id'])['exact_fill_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_core_copy['lag_fill_date_flag'] = sp_core_copy.groupby(['sp_patient_id', 'matched_patient_id'])['lag_fill_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_core_copy['days_supply_final_flag'] = sp_core_copy.groupby(['sp_patient_id', 'matched_patient_id'])['days_supply_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_core_copy['quantity_final_flag'] = sp_core_copy.groupby(['sp_patient_id', 'matched_patient_id'])['quantity_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_core_copy

C:\Users\RuchirTyagi\AppData\Local\Temp\ipykernel_28544\4193271119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_core_copy['rx_date_match_flag'] = sp_core_copy.groupby(['sp_patient_id', 'matched_patient_id'])['rx_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
C:\Users\RuchirTyagi\AppData\Local\Temp\ipykernel_28544\4193271119.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_core_copy['exact_fill_date_flag'] = sp_core_copy.groupby(['sp_patient_id', 'matched_patient_id'])['exact_fill_fla

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,rx_written_date,...,rx_flag,exact_fill_flag,lag_fill_flag,days_supply_flag,quantity_flag,rx_date_match_flag,exact_fill_date_flag,lag_fill_date_flag,days_supply_final_flag,quantity_final_flag
2,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,2025-08-05,...,0,-1,-1,-1,-1,0,0,0,0,0
25,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,2025-08-05,...,-1,-1,-1,-1,-1,0,0,0,0,0
37,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,2025-09-02,...,-1,-1,-1,-1,-1,0,0,0,0,0
49,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,2025-09-02,...,-1,-1,-1,-1,-1,0,0,0,0,0
61,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,2025-09-02,...,-1,-1,-1,-1,-1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38192,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,2025-09-03,...,-1,-1,-1,-1,-1,0,0,0,0,0
38206,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,2025-09-03,...,-1,-1,-1,-1,-1,0,0,0,0,0
38220,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,2025-09-03,...,-1,-1,-1,-1,-1,0,0,0,0,0
38234,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,2025-09-03,...,-1,-1,-1,-1,-1,0,0,0,0,0


In [387]:
sp_core_copy = sp_core_copy.drop_duplicates(subset=['sp_patient_id', 'matched_patient_id'])
sp_core_copy = sp_core_copy[['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state', 'rx_date_match_flag', 'exact_fill_date_flag', 'lag_fill_date_flag', 'days_supply_final_flag', 'quantity_final_flag']]
sp_core_copy

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,rx_date_match_flag,exact_fill_date_flag,lag_fill_date_flag,days_supply_final_flag,quantity_final_flag
2,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,0,0,0,0,0
25,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,0,0,0,0,0
37,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,0,0,0,0,0
49,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,0,0,0,0,0
61,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38192,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,0,0,0,0,0
38206,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,0,0,0,0,0
38220,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,0,0,0,0,0
38234,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,0,0,0,0,0


In [388]:
sp_diagnosis_therapy_df = pd.merge(sp_diagnosis_df, sp_core_copy, how='left', 
                        left_on = ['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state'],
                        right_on = ['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state'])
sp_diagnosis_therapy_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,icd_code,...,diag_freq,90_days_lookback_freq,180_days_lookback_freq,360_days_lookback_freq,first_last_claim_lookback,rx_date_match_flag,exact_fill_date_flag,lag_fill_date_flag,days_supply_final_flag,quantity_final_flag
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,C719,...,57,23,37,57,283.0,0.0,0.0,0.0,0.0,0.0
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,C719,...,89,2,2,9,679.0,0.0,0.0,0.0,0.0,0.0
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,C719,...,5,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,C719,...,41,2,3,41,158.0,0.0,0.0,0.0,0.0,0.0
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,C719,...,390,29,29,48,590.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2201,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,C719,...,5,5,5,5,105.0,0.0,0.0,0.0,0.0,0.0
2202,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,C719,...,21,1,1,9,721.0,0.0,0.0,0.0,0.0,0.0
2203,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,C719,...,2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2204,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,C719,...,0,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0


In [389]:
sp_payor_core_df = sp_payor_core_df[['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state', 'payor_name_flag', 'payor_type_flag', 'pbm_flag']]
sp_payor_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,payor_name_flag,payor_type_flag,pbm_flag
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,0,0,0
268,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,0,0,0
444,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,0,0,0
780,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,0,0,0
896,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
352966,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,0,1,0
353328,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,0,1,0
353348,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,0,1,0
353402,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,0,1,1


In [390]:
sp_payor_core_df['payor_name_final_flag'] = sp_payor_core_df.groupby(['sp_patient_id', 'matched_patient_id'])['payor_name_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_payor_core_df['payor_type_final_flag'] = sp_payor_core_df.groupby(['sp_patient_id', 'matched_patient_id'])['payor_type_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_payor_core_df['pbm_final_flag'] = sp_payor_core_df.groupby(['sp_patient_id', 'matched_patient_id'])['pbm_flag'].transform(lambda x: 1 if (x == 1).any() else 0)
sp_payor_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,payor_name_flag,payor_type_flag,pbm_flag,payor_name_final_flag,payor_type_final_flag,pbm_final_flag
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,0,0,0,0,0,0
268,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,0,0,0,0,0,0
444,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,0,0,0,0,0,0
780,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,0,0,0,0,0,0
896,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352966,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,0,1,0,0,1,0
353328,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,0,1,0,0,1,0
353348,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,0,1,0,0,1,0
353402,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,0,1,1,0,1,1


In [391]:
sp_payor_core_df = sp_payor_core_df[['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state', 'payor_name_final_flag', 'payor_type_final_flag', 'pbm_final_flag']]
sp_payor_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,payor_name_final_flag,payor_type_final_flag,pbm_final_flag
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,0,0,0
268,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,0,0,0
444,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,0,0,0
780,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,0,0,0
896,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
352966,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,0,1,0
353328,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,0,1,0
353348,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,0,1,0
353402,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,0,1,1


In [392]:
sp_payor_core_df = sp_payor_core_df.drop_duplicates(subset=['sp_patient_id', 'matched_patient_id'])
sp_payor_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,payor_name_final_flag,payor_type_final_flag,pbm_final_flag
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,0,0,0
268,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,0,0,0
444,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,0,0,0
780,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,0,0,0
896,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
352966,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,0,1,0
353328,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,0,1,0
353348,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,0,1,0
353402,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,0,1,1


In [393]:
sp_diagnosis_therapy_df = pd.merge(sp_diagnosis_therapy_df, sp_payor_core_df, how='left', 
                            left_on = ['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state'],
                        right_on = ['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state'])

sp_diagnosis_therapy_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,icd_code,...,360_days_lookback_freq,first_last_claim_lookback,rx_date_match_flag,exact_fill_date_flag,lag_fill_date_flag,days_supply_final_flag,quantity_final_flag,payor_name_final_flag,payor_type_final_flag,pbm_final_flag
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,C719,...,57,283.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,C719,...,9,679.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,C719,...,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,C719,...,41,158.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,C719,...,48,590.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2201,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,C719,...,5,105.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2202,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,C719,...,9,721.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2203,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,C719,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2204,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,C719,...,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [394]:
sp_diagnosis_therapy_df.to_excel(r'C:\Users\RuchirTyagi\OneDrive - ProcDNA Analytics Pvt. Ltd\Documents\Internal\Data Stitching\final_table.xlsx', index=False)

In [395]:
sp_core_df = pd.read_sql(sp_core_base_query, engine)
sp_core_df = sp_core_df.drop_duplicates(subset=['sp_prescriber_npi','sp_patient_id', 'matched_patient_id'])
sp_core_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI
...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA
